In [109]:
import requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [110]:
# Getting postalCode data from wikipedia link
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
labels=[]
data=[]
for tx in My_table.find_all('th'):
         labels.append(tx.text.strip())
rows = My_table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if cols:
     data.append([ele for ele in cols if ele])

In [111]:
# removing  rows where we have Not assigned data
df = pd.DataFrame.from_records(data, columns=labels)
df.rename(columns={'Postcode': 'PostalCode'}, inplace=True)
df1=df[df.Borough != 'Not assigned']
df1.shape

(212, 3)

In [112]:
#  updating rows where Neighbourhood is not assigned
for index, row in df1.iterrows():
     if (row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough']

In [113]:
# reading co-ordinates from Geospatial_Coordinates.csv file for each postal code and creating a new dataframe
import sys
import types
import pip

if not('ibm-cos-sdk' in [package.project_name for package in pip.get_installed_distributions()]):
    !pip install ibm-cos-sdk==2.0.0 -q

import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_63009217aad749368c287a2b914cbff9 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GvA3fdd2BSZEgS-8td33u1F-z92vvb-T3lTnAKf-C-e5',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_63009217aad749368c287a2b914cbff9.get_object(Bucket='datavisual-donotdelete-pr-2gfq67enauernl',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geodata = pd.read_csv(body)
geodata.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
result = pd.merge(df1,geodata,on="PostalCode")
result.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [114]:
# Performing clustering using K-MEAN and creating 5 clusters based on latitute and longitude to divide Toronto ,
#and then displaying on map each cluster with different colour

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from urllib.parse import urljoin
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

df_grouped = result.groupby('Neighbourhood').mean().reset_index()

kclusters = 5

df_clustering = df_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

k_means_cluster_centers = kmeans.cluster_centers_

df_merged = df_clustering

# add clustering labels
df_merged['Cluster Labels'] = kmeans.labels_


latitude=43.6542599
longitude=-79.3606359

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

#set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#print (folium.__version__)


# add markers to the map
#markers_colors = []
for lat, lon, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [9]:
#Creating a dataset containg postsalCode, neighbourhood and cluster number 
df_merged['Neighbourhood'] = df_grouped['Neighbourhood']
toronto_clustered = pd.merge(result,df_merged,on=['Neighbourhood','Latitude','Longitude'])
toronto_clustered.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,3
1,M4A,North York,Victoria Village,43.725882,-79.315572,2
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0
4,M6A,North York,Lawrence Heights,43.718518,-79.464763,3


In [115]:
#Calling FourSquare API to get location data for each venue ,
# we will be taking venue count and using it to decide most prominent venue and then deciding pizza store locations on basis of that

CLIENT_ID = 'T100LAO1E4PPVOWA2HPGLEMG0W4DV22XNKGMHIDD0RKE0XOC' # your Foursquare ID
CLIENT_SECRET = 'IYD1RDNHDYF5LKFJDJSU54XMX2ECMIPO4KFD1KZEEZOJADZY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

def getNearbyVenues(names,latitudes,longitudes, PostalCode,Borough,ClusterLabels,radius=500):
    
    venues_list=[]
    for name, lat, lng,pc,borough,cluster in zip(names,latitudes,longitudes,PostalCode,Borough,ClusterLabels):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            pc,
            borough,
            cluster,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'PostalCode',
                  'Borough',
                  'ClusterLabels',
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [116]:
toronto_venues = getNearbyVenues(names=toronto_clustered['Neighbourhood'],
                                   latitudes=toronto_clustered['Latitude'],
                                   longitudes=toronto_clustered['Longitude'],
                                   PostalCode=toronto_clustered['PostalCode'],
                                   Borough=toronto_clustered['Borough'],
                                   ClusterLabels=toronto_clustered['Cluster Labels']
                                  ) 

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto

In [16]:
toronto_venues.shape

(4258, 10)

In [120]:
# get the venue count in each neighbourhood
print(toronto_venues.shape)
toronto_venues.groupby(['Neighbourhood']).count()

(4258, 10)


,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Adelaide,100,100,100,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4,4,4,4
Agincourt North,2,2,2,2,2,2,2,2,2
Albion Gardens,11,11,11,11,11,11,11,11,11
Alderwood,10,10,10,10,10,10,10,10,10
Bathurst Manor,18,18,18,18,18,18,18,18,18
Bathurst Quay,14,14,14,14,14,14,14,14,14
Bayview Village,4,4,4,4,4,4,4,4,4
Beaumond Heights,11,11,11,11,11,11,11,11,11


In [121]:
# Number of venues in Cluster 0 locations
Toronto_Zone1=toronto_venues.loc[toronto_venues['ClusterLabels'] == 0].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)
Toronto_Zone1.head(20)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Adelaide,100,100,100,100,100,100,100,100,100
Design Exchange,100,100,100,100,100,100,100,100,100
Victoria Hotel,100,100,100,100,100,100,100,100,100
Union Station,100,100,100,100,100,100,100,100,100
Underground city,100,100,100,100,100,100,100,100,100
Toronto Islands,100,100,100,100,100,100,100,100,100
Toronto Dominion Centre,100,100,100,100,100,100,100,100,100
Ryerson,100,100,100,100,100,100,100,100,100
Richmond,100,100,100,100,100,100,100,100,100


Since in cluster 0  we have multiple locations with  venue 100 , locations having maximum pizza places

In [122]:
#locations having maximum pizza places in Cluster 0
Toronto_pizza_zone1=toronto_venues.loc[toronto_venues['ClusterLabels'] == 0]
Toronto_pizza_zone1=Toronto_pizza_zone1[Toronto_pizza_zone1['Venue Category'].str.contains("Pizza")]
Toronto_pizza_zone1.groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False).head(5)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Union Station,4,4,4,4,4,4,4,4,4
Toronto Islands,4,4,4,4,4,4,4,4,4
Harbourfront East,4,4,4,4,4,4,4,4,4
Little Portugal,2,2,2,2,2,2,2,2,2
Garden District,2,2,2,2,2,2,2,2,2


Three places identified in Cluster 0   are 'Union Station' , 'Toronto Islands' ,'Harbourfront East'

In [52]:
# Number of venues in Cluster 1
toronto_venues.loc[toronto_venues['ClusterLabels'] == 1].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Swansea,42,42,42,42,42,42,42,42,42
High Park,23,23,23,23,23,23,23,23,23
The Junction South,23,23,23,23,23,23,23,23,23
Roncesvalles,14,14,14,14,14,14,14,14,14
New Toronto,14,14,14,14,14,14,14,14,14
Mimico South,14,14,14,14,14,14,14,14,14
Humber Bay Shores,14,14,14,14,14,14,14,14,14
Parkdale,14,14,14,14,14,14,14,14,14
Royal York South West,13,13,13,13,13,13,13,13,13


Three places identified in Cluster1  are 'Swansea' , 'High Park' ,'The Junction South'

In [20]:
# Number of venues in Cluster 2
toronto_venues.loc[toronto_venues['ClusterLabels'] == 2].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Woodbine Gardens,13,13,13,13,13,13,13,13,13
Parkview Hill,13,13,13,13,13,13,13,13,13
Steeles West,12,12,12,12,12,12,12,12,12
L'Amoreaux West,12,12,12,12,12,12,12,12,12
Clairlea,9,9,9,9,9,9,9,9,9
Clarks Corners,9,9,9,9,9,9,9,9,9
Oakridge,9,9,9,9,9,9,9,9,9
Sullivan,9,9,9,9,9,9,9,9,9
Golden Mile,9,9,9,9,9,9,9,9,9


In [125]:
# Number of venues in Cluster 2
Toronto_Zone2=toronto_venues.loc[toronto_venues['ClusterLabels'] == 2]
Toronto_Zone2=Toronto_Zone2[Toronto_Zone2['Venue Category'].str.contains("Pizza")]
Toronto_Zone2.groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Clarks Corners,2,2,2,2,2,2,2,2,2
Parkview Hill,2,2,2,2,2,2,2,2,2
Sullivan,2,2,2,2,2,2,2,2,2
Tam O'Shanter,2,2,2,2,2,2,2,2,2
Woodbine Gardens,2,2,2,2,2,2,2,2,2
Guildwood,1,1,1,1,1,1,1,1,1
L'Amoreaux West,1,1,1,1,1,1,1,1,1
Morningside,1,1,1,1,1,1,1,1,1
Steeles West,1,1,1,1,1,1,1,1,1


Places identified in Cluster2 is 'Parkview Hill' since it has 2 pizza places and in this cluster since neighbourhood having mazimum venues is less than 20 so we would choose only 1 neighbourhood

In [21]:
# Number of venues in Cluster 3
toronto_venues.loc[toronto_venues['ClusterLabels'] == 3].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Fairview,64,64,64,64,64,64,64,64,64
Oriole,64,64,64,64,64,64,64,64,64
Henry Farm,64,64,64,64,64,64,64,64,64
Willowdale South,33,33,33,33,33,33,33,33,33
Lawrence Manor East,26,26,26,26,26,26,26,26,26
Bedford Park,26,26,26,26,26,26,26,26,26
Don Mills South,21,21,21,21,21,21,21,21,21
Flemingdon Park,21,21,21,21,21,21,21,21,21
North Toronto West,21,21,21,21,21,21,21,21,21


Three places identified in Cluster3  are 'Fairview' , 'Oriole' ,'Henry Farm'

In [22]:
# Number of venues in Cluster 4
toronto_venues.loc[toronto_venues['ClusterLabels'] == 4].groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Albion Gardens,11,11,11,11,11,11,11,11,11
Humbergate,11,11,11,11,11,11,11,11,11
Thistletown,11,11,11,11,11,11,11,11,11
South Steeles,11,11,11,11,11,11,11,11,11
Silverstone,11,11,11,11,11,11,11,11,11
Beaumond Heights,11,11,11,11,11,11,11,11,11
Jamestown,11,11,11,11,11,11,11,11,11
Mount Olive,11,11,11,11,11,11,11,11,11
Westmount,7,7,7,7,7,7,7,7,7


for cluster 4 since there are more than 3 places with 11 venue , we now need to see out of these 11 which has max pizza places 

In [103]:
# Number of venues in Cluster 4
Toronto_Zone4=toronto_venues.loc[toronto_venues['ClusterLabels'] == 4]
Toronto_Zone4=Toronto_Zone4[Toronto_Zone4['Venue Category'].str.contains("Pizza")]
Toronto_Zone4.groupby(['Neighbourhood']).count().sort_values(['Venue'],ascending=False)

,PostalCode,Borough,ClusterLabels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,,,,
Westmount,2,2,2,2,2,2,2,2,2
Albion Gardens,1,1,1,1,1,1,1,1,1
Beaumond Heights,1,1,1,1,1,1,1,1,1
Humber Summit,1,1,1,1,1,1,1,1,1
Humbergate,1,1,1,1,1,1,1,1,1
Jamestown,1,1,1,1,1,1,1,1,1
Kingsview Village,1,1,1,1,1,1,1,1,1
Martin Grove Gardens,1,1,1,1,1,1,1,1,1
Mount Olive,1,1,1,1,1,1,1,1,1


Place identified in Cluster4 is 'Westmount'

Summary:
    
Places identified in Cluster0   are 'Union Station' , 'Toronto Islands' ,'Harbourfront East'
<br>
Places identified in Cluster1 are 'Swansea' , 'High Park' ,'The Junction South'
<br>
Place identified in Cluster2 is 'Parkview Hill'
<br>
Places identified in Cluster3  are 'Fairview' , 'Oriole' ,'Henry Farm'
<br>
Place  identified in Cluster4 is 'Westmount'
